In [2]:
import json

for key in ['train', 'dev', 'test']:
    docs = []
    data = open('../data/datasets/scierc/raw_%s.json' % key)
    lines = data.readlines()
    for line in lines:
        docs.append(json.loads(line))
    print(len(lines))
    json.dump(docs, open('../data/datasets/scierc/raw_%s_valid.json' % key, 'w'))

350
50
100


In [3]:
import json

for key in ['train', 'dev', 'test']:

    cur_data = json.load(open('../data/datasets/scierc/scierc_%s.json' % key))
    raw_data = json.load(open('../data/datasets/scierc/raw_%s_valid.json' % key))

    cur_sent_idx = -1
    sent_doc_ids = []
    for doc_idx, doc in enumerate(raw_data):
        sent_doc_ids.extend(len(doc['sentences']) * [doc_idx])

    print(len(sent_doc_ids), len(cur_data))
    # print(sent_doc_ids)

    new_data = {}
    for sent_idx, sent in enumerate(cur_data):
        cur_doc_id = sent_doc_ids[sent_idx]
        if cur_doc_id not in new_data:
            new_data[cur_doc_id] = []
        new_data[cur_doc_id].append(sent)

    new_data_doc = []

    for doc_id in range(len(new_data)):
        word_shifts = [0]
        ent_shifts = [0]
        cur_doc = {}
        tokens = []
        entities = []
        relations = []
        for sent_idx, sent in enumerate(new_data[doc_id]):
            tokens.append(sent['tokens'])
            for ent in sent['entities']:
                ent['start'] += word_shifts[-1]
                ent['end'] += word_shifts[-1]
                ent['sent_id'] = sent_idx
                entities.append(ent)
            for rel in sent['relations']:
                rel['head'] += ent_shifts[-1]
                rel['tail'] += ent_shifts[-1]
                rel['sent_id'] = sent_idx
                relations.append(rel)

            word_shifts.append(word_shifts[-1]+len(sent['tokens']))
            ent_shifts.append(ent_shifts[-1]+len(sent['entities']))
        
        new_data_doc.append({
            'tokens': tokens, 'entities': entities, 'relations': relations})

    json.dump(new_data_doc, open('../data/datasets/scierc/scierc_%s_new_doc.json' % key, 'w'))

1861 1861
275 275
551 551


Check inconsistency in dataset

In [12]:
import json

# for key in ['train', 'dev', 'test']:
key = 'test'
data_doc = json.load(open('../data/datasets/scierc/scierc_%s_new_doc.json' % key))

inconsistence = []

for doc_idx, doc in enumerate(data_doc):
    types = {}
    for ent in doc['entities']:
        word = doc['tokens'][ent['sent_id']][ent['start']: ent['end']]
        word = ' '.join(word)
        if word not in types:
            types[word] = []
        types[word].append(ent['type'])
    
    for word, keys in types.items():
        if len(set(keys)) != 1:
            inconsistence.append({'doc': doc_idx, 'word': word, 'types': types})


print(len(inconsistence))
for record in inconsistence:
    # print(record['doc'])
    for word, keys in record['types'].items():
        if len(set(keys)) != 1:
            print(word, set(keys))

98
 {'Generic', 'OtherScientificTerm', 'Material', 'Method', 'Task'}
 {'OtherScientificTerm', 'Generic', 'Method', 'Task'}
 {'Generic', 'OtherScientificTerm', 'Material', 'Method', 'Task'}
 {'Material', 'Method', 'Generic', 'Task'}
 {'OtherScientificTerm', 'Method', 'Generic', 'Task'}
 {'Generic', 'Metric', 'OtherScientificTerm', 'Method', 'Task'}
 {'Metric', 'Generic', 'OtherScientificTerm', 'Material', 'Method', 'Task'}
 {'Metric', 'Generic', 'Method'}
 {'OtherScientificTerm', 'Material', 'Method', 'Task'}
 {'OtherScientificTerm', 'Generic', 'Method'}
 {'Generic', 'Metric', 'OtherScientificTerm', 'Material', 'Method'}
 {'OtherScientificTerm', 'Method', 'Generic', 'Task'}
 {'OtherScientificTerm', 'Method', 'Task'}
 {'Generic', 'Metric', 'OtherScientificTerm', 'Material', 'Method'}
 {'Generic', 'Metric', 'OtherScientificTerm', 'Material', 'Method', 'Task'}
 {'Generic', 'Metric', 'OtherScientificTerm', 'Method', 'Task'}
 {'OtherScientificTerm', 'Generic', 'Method', 'Task'}
 {'OtherScien

In [7]:
import json

data = json.load(open('../data/datasets/scierc/scierc_test_new_doc.json'))
preds = json.load(open('../data/log/scierc_train/doc_baseline/predictions_valid_epoch_27.json'))


inconsistence = []
all_records = []

for doc_idx, doc in enumerate(preds):
    tokens = []
    for ts in data[doc_idx]['tokens']:
        tokens.extend(ts)

    types = {}
    for ent in doc['entities']:
        word = tokens[ent['start']: ent['end']]
        word = ' '.join(word)
        if word not in types:
            types[word] = []
        types[word].append(ent['type'])
    
    for word, keys in types.items():
        if len(set(keys)) != 1:
            inconsistence.append({'doc': doc_idx, 'types': types})
        all_records.append({'doc': doc_idx, 'types': types})


print(len(inconsistence))
for record in inconsistence:
    print(record['doc'])
    for word, keys in record['types'].items():
        if len(set(keys)) != 1:
            print(word, set(keys))

mentions = []
for record in all_records:
    for keys in record['types'].values():
        mentions.append(keys)

print('entities with mentions > 1: ', len([keys for keys in mentions if len(keys)>1]))
print('average number of mentions: ', sum(len(keys) for keys in mentions) /len(mentions))

5
16
robust PCA {'Method', 'Task'}
61
PER {'OtherScientificTerm', 'Metric'}
69
discourse {'OtherScientificTerm', 'Material'}
75
word list {'OtherScientificTerm', 'Material'}
94
text input {'OtherScientificTerm', 'Material'}
entities with mentions > 1:  1918
average number of mentions:  1.1025418615175744


In [4]:
import json

train_data = json.load(open('../data/datasets/scierc/scierc_train_new_doc.json'))
dev_data = json.load(open('../data/datasets/scierc/scierc_dev_new_doc.json'))

train_data.extend(dev_data)

json.dump(train_data, open('../data/datasets/scierc/scierc_tain_dev_new_doc.json', 'w'))